### Calculator

In [ ]:

import openpyxl
import numpy as np
import copy
import math

##read data from Excel
wb = openpyxl.load_workbook('test_calculator1.xlsx',data_only=True)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
## for temporary test
'''
coe ={}
sheet_Profile1 = wb['Profile1']
first_column = sheet_Profile1['AG']

# Print the contents
for x in range(2,6): 
  print(first_column[x].value) 
'''

2.4632
1.561
-0.9545
-1.3055


In [ ]:
def readKidneyData(wb):
  ## read data related to kidney
  sheet_Input = wb['Profile Input']
  sheet_Profile1 = wb['Profile1']

  ##read low, regular, waiting time
  low_function = [sheet_Input['B4'].value,sheet_Input['C4'].value]
  regular_function = [sheet_Input['B5'].value,sheet_Input['C5'].value]
  waiting = [sheet_Input['B7'].value,sheet_Input['C7'].value]

  ## read coefficients (should I read the original data??)
  coe_kidney={}
  coe_kidney[8,5]=sheet_Profile1['C2'].value
  coe_kidney[5,5]=sheet_Profile1['C3'].value
  coe_kidney[3,5]=sheet_Profile1['C4'].value
  coe_kidney[2,5]=sheet_Profile1['C5'].value

  coe_kidney[8,3]=sheet_Profile1['F2'].value
  coe_kidney[5,3]=sheet_Profile1['F3'].value
  coe_kidney[3,3]=sheet_Profile1['F4'].value
  coe_kidney[2,3]=sheet_Profile1['F5'].value

  coe_kidney[8,2]=sheet_Profile1['I2'].value
  coe_kidney[5,2]=sheet_Profile1['I3'].value
  coe_kidney[3,2]=sheet_Profile1['I4'].value
  coe_kidney[2,2]=sheet_Profile1['I5'].value

  coe_kidney['waiting']=sheet_Profile1['L2'].value

  return low_function,regular_function,waiting,coe_kidney


In [ ]:
def kidney_preference(coe,low_function,regular_function,waiting):
  time_list = [8,5,3,2]
  degree = 4
  preference_weight=[]
  for j in range(2):
    ##Generating weights for regular function based on low function level selected for profile 
    coe_between={}
    for t in time_list:
      if low_function[j] >3:
        coe_between[t,3,5] = coe[t,5]*(low_function[j]-3)/2+coe[t,3]*(1-(low_function[j]-3)/2)
        coe_between[t,2,3] = 0
      else:
        coe_between[t,3,5] = 0
        coe_between[t,2,3] = coe[t,3]*(low_function[j]-2)+coe[t,2]*(1-(low_function[j]-2))
    Y_35 = []
    Y_23 = []
    for t in time_list:
      Y_35.append(coe_between[t,3,5])
      Y_23.append(coe_between[t,2,3])
    ## beta of trendlines
    X_matrix = []
    for i in time_list:
      X_row = []
      for d in range(degree):
        X_row.append(i**d)
      X_matrix.append(X_row)
    beta_23 = np.dot(np.linalg.inv(np.dot(np.transpose(X_matrix),X_matrix)),np.dot(np.transpose(X_matrix),Y_23))
    beta_35 = np.dot(np.linalg.inv(np.dot(np.transpose(X_matrix),X_matrix)),np.dot(np.transpose(X_matrix),Y_35))
    ## generate preference weight for the specific regular function, waiting and low function
    preference_weight.append(sum(beta_23[i]*regular_function[j]**i for i in range(degree)) + sum(beta_35[i]*regular_function[j]**i for i in range(degree)) + waiting[j]*coe['waiting'])
  
  ## calculate the probability of accepting one kidney compared to another
  accept_probability_kidney_1 = math.exp(preference_weight[0]-preference_weight[1])/(1+math.exp(preference_weight[0]-preference_weight[1]))
  accept_probability_kidney = [accept_probability_kidney_1,1- accept_probability_kidney_1]
  return preference_weight,accept_probability_kidney

In [ ]:
low_function,regular_function,waiting,coe_kidney = readKidneyData(wb)
preference_weight,accept_probability_kidney = kidney_preference(coe_kidney,low_function,regular_function,waiting)
print("probability of accepting kidney 1:", accept_probability_kidney[0])
print("probability of accepting kidney 2:", accept_probability_kidney[1])

probability of accepting kidney 1: 0.1349690388077945
probability of accepting kidney 2: 0.8650309611922055


In [ ]:
def readPysicianData(wb):
  ## read data related to physician
  sheet_Input = wb['Profile Input']
  sheet_PhysicianInfo = wb['Physician information']
  coe={}
  coe['ejection']=[sheet_PhysicianInfo['C11'].value,sheet_PhysicianInfo['C12'].value]
  coe['hla']= [sheet_PhysicianInfo['C14'].value,sheet_PhysicianInfo['C15'].value]
  coe['sensitization'] = [sheet_PhysicianInfo['C17'].value,sheet_PhysicianInfo['C18'].value,sheet_PhysicianInfo['C19'].value]
  coe['age']=[sheet_PhysicianInfo['C25'].value,sheet_PhysicianInfo['C26'].value,sheet_PhysicianInfo['C27'].value]
  coe['dialysis']=[sheet_PhysicianInfo['C29'].value,sheet_PhysicianInfo['C30'].value,sheet_PhysicianInfo['C31'].value]
  coe['coldtime'] = sheet_PhysicianInfo['C6'].value

  para_list = {}
  para_list['ejection']=[sheet_PhysicianInfo['B11'].value,sheet_PhysicianInfo['B12'].value]
  para_list['hla'] = [sheet_PhysicianInfo['D40'].value,sheet_PhysicianInfo['D41'].value] ##upper table is string, so read the lower table
  para_list['sensitization'] = [sheet_PhysicianInfo['B17'].value,sheet_PhysicianInfo['B18'].value,sheet_PhysicianInfo['B19'].value]
  para_list['age'] = [sheet_PhysicianInfo['B25'].value,sheet_PhysicianInfo['B26'].value,sheet_PhysicianInfo['B27'].value]
  para_list['dialysis'] = [sheet_PhysicianInfo['D51'].value,sheet_PhysicianInfo['D52'].value,sheet_PhysicianInfo['D53'].value]##upper table is string, so read the lower table


  physican_input = {}
  physican_input['ejection']=sheet_Input['B16'].value
  physican_input['hla'] = sheet_Input['B17'].value
  physican_input['sensitization'] = sheet_Input['B18'].value
  physican_input['age'] = sheet_Input['B15'].value
  physican_input['dialysis'] = sheet_Input['B19'].value

  ##input that no need OLS
  physican_input['KDPI'] = [sheet_Input['F5'].value,sheet_Input['G5'].value]
  physican_input['diabetes'] = sheet_Input['B13'].value
  physican_input['performance'] = sheet_Input['B14'].value[0] #cannot compare the string with changed lines, so here I just compare the first letter

  return coe,para_list, physican_input


In [ ]:
def physician_preference(coe,para_list,physican_input): 
  category_list = ['ejection','hla','sensitization','age','dialysis']
  degree_list = {}
  degree_list['ejection']=2
  degree_list['hla'] = 2
  degree_list['sensitization'] = 3
  degree_list['age'] = 3
  degree_list['dialysis'] = 3
  beta = {}
  preference_weight = {}
  ## calculate preference weights for some categories
  for i in category_list:
    X_matrix = []
    for j in range(degree_list[i]):
      X_row = []
      for d in range(degree_list[i]):
        X_row.append(para_list[i][j]**d)
      X_matrix.append(X_row)
    beta[i] = np.dot(np.linalg.inv(np.dot(np.transpose(X_matrix),X_matrix)),np.dot(np.transpose(X_matrix),coe[i]))
    preference_weight[i]= sum(beta[i][d]*physican_input[i]**d for d in range(degree_list[i]))
  
  ## read preference weights that don't need OLS
  for j in range(2):
    if physican_input['KDPI'][j]>7:
      preference_weight['KDPI',j]=2.617
    else:
      preference_weight['KDPI',j]=-1.041
  

  if physican_input['diabetes'] == 'No diabetes':
    preference_weight['diabetes']= 2.768
  if physican_input['diabetes'] == 'Yes, but not insulin dependent':
    preference_weight['diabetes']= 2.647
  if physican_input['diabetes'] == 'Insulin dependent, but less than 10 years':
    preference_weight['diabetes']= -5.415

  if physican_input['performance'] == 'N':
    preference_weight['performance']= 0.062
  if physican_input['performance'] == 'R':
    preference_weight['performance']= -0.195
  if physican_input['performance'] == 'C':
    preference_weight['performance']= 0.133
  
  ## calculate weights by sum up weights from different categories
  preference_weight_all = [sum(preference_weight[i] for i in category_list)+ preference_weight['KDPI',j]+preference_weight['diabetes']+preference_weight['performance'] for j in range(2)]
  preference_weight_20cold = [preference_weight_all[j] + 20*coe['coldtime'] for j in range(2)]
  ## calculate acceptance probability
  acceptance = [math.exp(preference_weight_all[j])/(math.exp(preference_weight_all[j])+1) for j in range(2)]
  acceptance_20cold = [math.exp(preference_weight_20cold[j])/(math.exp(preference_weight_20cold[j])+1) for j in range(2)]

  return acceptance,acceptance_20cold

In [ ]:
coe,para_list,physican_input = readPysicianData(wb)
acceptance,acceptance_20cold=physician_preference(coe,para_list,physican_input)
print("Chance of acceptance with 20 hrs of cold ischemia time of kidney 2:",acceptance_20cold[1]) ## Excel only reports the kidney accpetance chance

Chance of acceptance with 20 hrs of cold ischemia time of kidney 2: 0.9610242845400692
